In [1]:
import os
import random
import time

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from datasets.user_profile_mode import UserProfileModeDataset
from models.acf import ACF
from trainers.acf_trainer import ACFTrainer
from trainers.losses import warp_loss

In [2]:
# Parameters
RNG_SEED = 0
EMBEDDING_PATH = os.path.join("data", "embedding-resnet50-layer4.npy")
TRAINING_PATH = os.path.join("data", "naive-user-train.csv")
VALIDATION_PATH = os.path.join("data", "naive-user-validation.csv")
CHECKPOINTS_DIR = os.path.join("checkpoints")
USE_GPU = True

# Parameters (training)
SETTINGS = {
    "batch_sampler:batch_size": 128,
    "optimizer:lr": 1e-3,
    "optimizer:weight_decay": 1e-5,
    "train:max_epochs": 10,
    "model:model_dim": 128,
}

In [3]:
%%time
# Freezing RNG seed if needed
if RNG_SEED is not None:
    print(f"\nUsing random seed...")
    random.seed(RNG_SEED)
    torch.manual_seed(RNG_SEED)
    np.random.seed(RNG_SEED)

# Training DataLoader
train_dataset = UserProfileModeDataset(
    csv_file=TRAINING_PATH,
)
print(f">> Training dataset: {len(train_dataset)}")

# Validation DataLoader
valid_dataset = UserProfileModeDataset(
    csv_file=VALIDATION_PATH,
)
print(f">> Validation dataset: {len(valid_dataset)}")

# Model initialization
print("\nInitialize model")
device = torch.device("cuda:0" if torch.cuda.is_available() and USE_GPU else "cpu")
if torch.cuda.is_available() != USE_GPU:
    print((f"\nNotice: Not using GPU - "
           f"Cuda available ({torch.cuda.is_available()}) "
           f"does not match USE_GPU ({USE_GPU})"
    ))
model = ACF(
    train_dataset.users,
    train_dataset.items,
    feature_path=EMBEDDING_PATH,
    model_dim=SETTINGS["model:model_dim"],
    device=device
).to(device)

# Training setup
print("\nSetting up training")
optimizer = optim.Adam(
    model.parameters(),
    lr=SETTINGS["optimizer:lr"],
    weight_decay=SETTINGS["optimizer:weight_decay"],
)
criterion = warp_loss

# Training
print("\nTraining")



Using random seed...
>> Training dataset: 5000086
>> Validation dataset: 500656

Initialize model

Setting up training

Training
CPU times: user 1min 6s, sys: 9.34 s, total: 1min 16s
Wall time: 1min 12s


In [5]:
%%time
# Training
version = (
    f"{model.__class__.__name__}_"
    f"wikimedia"
)

datasets = train_dataset, valid_dataset
trainer = ACFTrainer(
    model, datasets, criterion, optimizer, version,
    device=device, batch_size=SETTINGS["batch_sampler:batch_size"],
    checkpoint_dir=CHECKPOINTS_DIR
)

# best_model, best_acc, best_loss, best_epoch = trainer.fit(SETTINGS["train:max_epochs"])
trainer.fit(SETTINGS["train:max_epochs"])

# # Final result
# print(f"\nBest ACC {best_acc} reached at epoch {best_epoch}")
# print(best_model)

  0%|          | 0/10 [00:00<?, ?it/s]



Starting epoch: 0 | phase: train | ⏰: 11:58:38


  0%|          | 0/39064 [00:02<?, ?it/s]

KeyboardInterrupt: 